# Robotics toolbox Interactive example using Swift backend
![](https://github.com/petercorke/robotics-toolbox-python/raw/master/docs/figs/RobToolBox_RoundLogoB.png)


## Installing the necessary packages
Install `jupyter` to run this notebook
```
pip install jupyter
```
Install the robotics toolbox
```
pip install roboticstoolbox-python
```
If you still have not installed `pycapacity`, you can install it using
```
pip install pycapacity
```

For interactive visualisation
```
pip install ipywidgets
pip install ipympl
```

### Swift does not remove old polytopes

This is a problem of Swift, a bugfix was sent in this pull request: https://github.com/jhavl/swift/pull/49<br>
Untill Swift has integrated the bugfix, please use this fork which has updated code: https://github.com/askuric/swift/tree/master<br>
You can simply install it with
```
pip install git+https://github.com/askuric/swift.git
```

In [1]:
import roboticstoolbox as rp
import numpy as np
import os

# variable containing the reference of the polytope
poly_mesh = None

In [2]:
import time
# polytope python module
import pycapacity.robot as pyc
# polytope visualisation
import trimesh
from spatialgeometry import Mesh

# funciton, calculating and visualising the polytope
def visualise_polytope(q, rob, env):
    Jac = rob.jacob0(q)[:3,:]
    # gravity torque
    gravity = rob.gravload(q).reshape((-1,1))
    
    # calculate for the polytope
    f_poly =  pyc.force_polytope(Jac, t_max, t_min, gravity)
    f_poly.find_faces()
    # save polytope as mesh file
    scaling = 500
    # create the mesh
    mesh = trimesh.Trimesh(vertices=(f_poly.vertices.T/scaling + rob.fkine(q).t),
                           faces=f_poly.face_indices, use_embree=True, validate=True)

    # remove the old polytope files generated
    file_dir = os.path.join(os.getcwd())
    for i in os.listdir(file_dir):
        if "tmp_polytope" in i:
            os.remove(os.path.join(file_dir,i))
    
    # save the new polytope file
    # in the stl format
    file = os.path.join(file_dir,'tmp_polytope_file{}.stl'.format(int(100*time.time())))
    f = open(file, "wb")
    f.write(trimesh.exchange.stl.export_stl(mesh))
    f.close()
    # create the Mesh class from the file
    # that is the only way to do it for the moment
    # that's a pitty
    poly_mesh = Mesh(file)
    poly_mesh.color = (0.9,0.6,0.0,0.5)
    # visualise the polytope
    env.add(poly_mesh)
    return poly_mesh

In [3]:
# robot for calculating the kinematics and jacobians
robot = rp.models.DH.Panda()
# initial pose
q= np.array([0.00138894 ,5.98736e-05,-0.30259058,   -1.6, -6.64181e-05,    1.56995,-5.1812e-05])
robot.q = q
q_min,q_max = robot.qlim
# joint torque limits
t_max = np.array([87, 87, 87, 87, 20, 20, 20])
t_min = -t_max


# visualisation robot
robot_vis = rp.models.Panda()
import swift.Swift as Swift
robot_vis.q = q
env = Swift()
env.launch(browser="notebook") # if having an old version of swift replace with env.launch()
env.add(robot_vis)

0

In [4]:
from ipywidgets import interact, FloatSlider
kwargs = {'q[{}]'.format(i) : 
          FloatSlider(
              min = q_min[i], 
              max = q_max[i], 
              step = 0.01, 
              value = q[i],
              continuous_update=False) 
          for i,q_1 in enumerate(q)}
@interact(**kwargs)
def update(**kwargs):
    global poly_mesh
    q = np.array([v  for v in kwargs.values()])
    robot_vis.q = q
    # remove the old polytope
    if poly_mesh: # if having an old version of remove this condition
        env.remove(poly_mesh)
    # plot the new polytope
    poly_mesh = visualise_polytope(q, robot, env)
    # robot visualisation
    env.step()

interactive(children=(FloatSlider(value=0.00138894, continuous_update=False, description='q[0]', max=2.8973, m…